In [ ]:
!pip install pandas numpy scikit-learn matplotlib seaborn

In [ ]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split  # For initial validation
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
import numpy as np

# Define the full column names (from the dataset description)
full_column_names = [
    'duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes',
    'land', 'wrong_fragment', 'urgent', 'hot', 'num_failed_logins',
    'logged_in', 'num_compromised', 'root_shell', 'su_attempted',
    'num_root', 'num_file_creations', 'num_shells', 'num_access_files',
    'num_outbound_cmds', 'is_host_login', 'is_guest_login', 'count',
    'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate',
    'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate',
    'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count',
    'dst_host_same_srv_rate', 'dst_host_diff_srv_rate',
    'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate',
    'dst_host_serror_rate', 'dst_host_srv_serror_rate',
    'dst_host_rerror_rate', 'dst_host_srv_rerror_rate',
    'attack_type', 'difficulty_score'
]


# Define the important features you want to use
important_features = [
    'duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes',
    'logged_in', 'count', 'srv_count', 'serror_rate', 'srv_serror_rate',
    'same_srv_rate', 'dst_host_count', 'dst_host_srv_count',
    'dst_host_same_srv_rate', 'dst_host_diff_srv_rate',
    'dst_host_serror_rate', 'dst_host_srv_serror_rate'
]

# --- 1. Load Training Data ---
train_file_path = 'KDDTrain+.txt'
try:
    df_train = pd.read_csv(train_file_path, header=None,
                           names=full_column_names)
    print(f"Successfully loaded {train_file_path}")
    print(f"Full training dataset shape: {df_train.shape}")
except FileNotFoundError:
    print(f"Error: {train_file_path} not found. Please upload the file.")
    exit()
except Exception as e:
    print(f"An error occurred while loading the training data: {e}")
    exit()

# --- 2. Prepare Training Data ---
# Extract features and target variable
X_train = df_train[important_features].copy()
y_train = df_train['attack_type'].apply(
    lambda x: 0 if x == 'normal' else 1)  # Binary: 0=normal, 1=attack

# Identify categorical and numerical features
categorical_features = X_train.select_dtypes(
    include=['object', 'category']).columns
numerical_features = X_train.select_dtypes(
    include=['int64', 'float64']).columns

print("\nTarget variable distribution (Train - Binary: 0=normal, 1=attack):")
print(y_train.value_counts())
print(f"\nShape of selected features (X_train): {X_train.shape}")
print(
    f"\nSelected Categorical features for OHE: {list(categorical_features)}")
print(f"Selected Numerical features: {list(numerical_features)}")

# --- 3. Create Preprocessor ---
# ColumnTransformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False),
         categorical_features)
    ],
    remainder='passthrough')  # Or 'drop' if you want to drop other columns

# --- 4. Build Model Pipeline ---
# Create a pipeline with preprocessing and LightGBM
model = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier',
                       lgb.LGBMClassifier(random_state=42,
                                        n_estimators=100, # Example: Tune!
                                        learning_rate=0.1, # Example: Tune!
                                        max_depth=10,     # Example: Tune!
                                        n_jobs=-1))])  # Use all cores

# --- 5. Train the Model ---
print("\nTraining the LightGBM model...")
model.fit(X_train, y_train)  # Train the pipeline
print("Model training complete.")

# --- 6. Load Test Data ---
test_file_path = 'KDDTest+.txt'
try:
    df_test = pd.read_csv(test_file_path, header=None,
                           names=full_column_names)
    print(f"\nSuccessfully loaded {test_file_path}")
    print(f"Full test dataset shape: {df_test.shape}")
except FileNotFoundError:
    print(
        f"Error: {test_file_path} not found. Please upload the file.")
    exit()
except Exception as e:
    print(f"An error occurred while loading the test data: {e}")
    exit()


# --- 7. Prepare Test Data ---
# Extract features and target variable from the test set
X_test = df_test[important_features].copy()  # Use the SAME features
y_test = df_test['attack_type'].apply(
    lambda x: 0 if x == 'normal' else 1)  # Binary: 0=normal, 1=attack


# --- 8. Preprocess Test Data ---
# ***CRITICAL STEP:*** Use the fitted preprocessor from training!
y_pred = model.predict(X_test)


# Print the classification report
print("\n--- Evaluation on KDDTest+.txt ---")
print(classification_report(y_test, y_pred,
                              target_names=['Normal', 'Attack']))

Successfully loaded KDDTrain+.txt
Full training dataset shape: (125973, 43)

Target variable distribution (Train - Binary: 0=normal, 1=attack):
attack_type
0    67343
1    58630
Name: count, dtype: int64

Shape of selected features (X_train): (125973, 18)

Selected Categorical features for OHE: ['protocol_type', 'service', 'flag']
Selected Numerical features: ['duration', 'src_bytes', 'dst_bytes', 'logged_in', 'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'same_srv_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate']

Training the LightGBM model...


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Number of positive: 58630, number of negative: 67343
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.022025 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2459
[LightGBM] [Info] Number of data points in the train set: 125973, number of used features: 90
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.465417 -> initscore=-0.138552
[LightGBM] [Info] Start training from score -0.138552
Model training complete.

Successfully loaded KDDTest+.txt
Full test dataset shape: (22544, 43)

--- Evaluation on KDDTest+.txt ---
              precision    recall  f1-score   support

      Normal       0.66      0.97      0.79      9711
      Attack       0.97      0.63      0.76     12833

    accuracy                           0.78     22544
   macro avg       0.82      0.80      0.78     22544
weighted avg       0.84      0.78    

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [ ]:
# --- 10. Batch Prediction from File ---
print("\n=== Batch Prediction from File ===")
print(f"Reading inputs from 'input_data.txt'")
print("Note: Lines starting with '#' and blank lines will be ignored")
print(f"Each valid line should contain {len(important_features)} comma-separated values")
print("Expected features:", ", ".join(important_features))

custom_input_path = 'input_data.txt'

try:
    with open(custom_input_path, 'r') as f:
        # Filter out comments and blank lines
        lines = [line.strip() for line in f
                if line.strip() and not line.strip().startswith('#')]

        if not lines:
            print("\n❌ Error: No valid data found in input file")
            print("(File is empty or contains only comments/blank lines)")
            exit()

        print(f"\nFound {len(lines)} valid records to process:\n")

        results = []
        valid_count = 0
        for i, line in enumerate(lines, 1):
            try:
                # Skip empty lines (additional check)
                if not line.strip():
                    continue

                # Skip comment lines (additional check)
                if line.strip().startswith('#'):
                    continue

                values = [x.strip() for x in line.split(',')]

                if len(values) != len(important_features):
                    print(f"{i}. ❌ SKIPPED: Expected {len(important_features)} values, got {len(values)}")
                    print(f"   Problematic line: {line[:80]}{'...' if len(line) > 80 else ''}")
                    continue

                # Convert to DataFrame
                input_df = pd.DataFrame([values], columns=important_features)

                # Type conversion
                for col in input_df.columns:
                    if col in numerical_features:
                        input_df[col] = pd.to_numeric(input_df[col])
                    elif col in categorical_features:
                        input_df[col] = input_df[col].astype(str)

                # Prediction
                prediction = model.predict(input_df)[0]
                proba = model.predict_proba(input_df)[0]

                valid_count += 1
                results.append({
                    'id': valid_count,  # Use sequential count of valid records
                    'prediction': 'Normal' if prediction == 0 else 'Attack',
                    'normal_conf': proba[0],
                    'attack_conf': proba[1],
                    'input': line[:50] + '...' if len(line) > 50 else line
                })

                print(f"{valid_count}. ✓ {results[-1]['prediction']} "
                      f"(Normal: {proba[0]:.2%}, "
                      f"Attack: {proba[1]:.2%})")

            except Exception as e:
                print(f"{i}. ❌ ERROR: {str(e)}")
                print(f"   Problematic line: {line[:80]}{'...' if len(line) > 80 else ''}")
                continue

        # Display summary
        if results:
            print("\n\n📊 Prediction Summary:")
            print("-" * 65)
            print(f"{'No.':<4} {'Prediction':<10} {'Normal Conf.':<15} {'Attack Conf.':<15} {'Sample Input':<20}")
            print("-" * 65)
            for res in results:
                print(f"{res['id']}.  {res['prediction']:<10} {res['normal_conf']:.2%}{'':<6} {res['attack_conf']:.2%}{'':<6} {res['input']}")

            # Statistics
            attack_count = sum(1 for res in results if res['prediction'] == 'Attack')
            print("\n🔍 Summary Statistics:")
            print(f"• Total valid records processed: {valid_count}/{len(lines)}")
            print(f"• Normal traffic detected: {valid_count - attack_count} ({(valid_count - attack_count)/valid_count:.1%})")
            print(f"• Attacks detected: {attack_count} ({attack_count/valid_count:.1%})")
        else:
            print("\n⚠️ No valid predictions were generated")

except FileNotFoundError:
    print(f"\n❌ Error: File '{custom_input_path}' not found")
    print("Please create an 'input_data.txt' file with your input data.")
except Exception as e:
    print(f"\n❌ Unexpected error: {e}")


=== Batch Prediction from File ===
Reading inputs from 'input_data.txt'
Note: Lines starting with '#' and blank lines will be ignored
Each valid line should contain 18 comma-separated values
Expected features: duration, protocol_type, service, flag, src_bytes, dst_bytes, logged_in, count, srv_count, serror_rate, srv_serror_rate, same_srv_rate, dst_host_count, dst_host_srv_count, dst_host_same_srv_rate, dst_host_diff_srv_rate, dst_host_serror_rate, dst_host_srv_serror_rate

Found 12 valid records to process:

1. ✓ Attack (Normal: 0.02%, Attack: 99.98%)
2. ✓ Attack (Normal: 0.02%, Attack: 99.98%)
3. ✓ Normal (Normal: 99.99%, Attack: 0.01%)


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11

4. ✓ Attack (Normal: 0.02%, Attack: 99.98%)
5. ✓ Normal (Normal: 99.85%, Attack: 0.15%)
6. ✓ Attack (Normal: 0.02%, Attack: 99.98%)


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11

7. ✓ Normal (Normal: 99.94%, Attack: 0.06%)
8. ✓ Normal (Normal: 70.34%, Attack: 29.66%)
9. ✓ Normal (Normal: 100.00%, Attack: 0.00%)
10. ✓ Normal (Normal: 90.17%, Attack: 9.83%)
11. ✓ Normal (Normal: 100.00%, Attack: 0.00%)
12. ✓ Normal (Normal: 71.88%, Attack: 28.12%)


📊 Prediction Summary:
-----------------------------------------------------------------
No.  Prediction Normal Conf.    Attack Conf.    Sample Input        
-----------------------------------------------------------------
1.  Attack     0.02%       99.98%       0,tcp,http,SF,0,0,0,123,6,1.00,1.00,0.05,255,26,0....
2.  Attack     0.02%       99.98%       0,tcp,http,S0,0,0,0,270,23,1.00,1.00,0.09,255,23,0...
3.  Normal     99.99%       0.01%       0,tcp,http,SF,232,8153,1,5,5,0.20,0.20,1.00,30,255...
4.  Attack     0.02%       99.98%       0,tcp,http,SF,0,0,0,123,6,1.00,1.00,0.05,255,26,0....
5.  Normal     99.85%       0.15%       0,tcp,ftp_data,SF,491,0,0,2,2,0.00,0.00,1.00,150,2...
6.  Attack     0.02%       99.98% 

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11

In [ ]:
# --- 10. Interactive Custom Input Prediction ---
print("\n=== Custom Input Prediction ===")
print("Enter values for the following features (comma-separated):")
print(", ".join(important_features))

# Get input from user
user_input = input("\nEnter values: ").strip()

try:
    # Process the input
    values = [x.strip() for x in user_input.split(',')]

    # Convert to DataFrame
    input_df = pd.DataFrame([values], columns=important_features)

    # Convert types to match training data
    for col in input_df.columns:
        if col in numerical_features:
            input_df[col] = pd.to_numeric(input_df[col])
        elif col in categorical_features:
            input_df[col] = input_df[col].astype(str)

    # Predict using the pipeline
    prediction = model.predict(input_df)[0]
    pred_label = 'Normal' if prediction == 0 else 'Attack'
    proba = model.predict_proba(input_df)[0]

    print(f"\n🔍 Prediction: {pred_label}")
    print(f"   Confidence: Normal: {proba[0]:.2%}, Attack: {proba[1]:.2%}")

except Exception as e:
    print(f"\n❌ Error processing input: {e}")
    print("Please make sure you entered all 18 values in the correct order.")


=== Custom Input Prediction ===
Enter values for the following features (comma-separated):
duration, protocol_type, service, flag, src_bytes, dst_bytes, logged_in, count, srv_count, serror_rate, srv_serror_rate, same_srv_rate, dst_host_count, dst_host_srv_count, dst_host_same_srv_rate, dst_host_diff_srv_rate, dst_host_serror_rate, dst_host_srv_serror_rate

Enter values: 0,tcp,http,SF,0,0,0,123,6,1.00,1.00,0.05,255,26,0.10,0.05,1.00,1.00

🔍 Prediction: Attack
   Confidence: Normal: 0.02%, Attack: 99.98%


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
